In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import os
import scipy
from skimage import io
import multiprocessing as mp
from itertools import repeat

In [2]:
datagen = ImageDataGenerator(
    rotation_range=20, #rotate between 0 and 20 degree
    width_shift_range=0.1, # shift width
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest', # constant, nearest, reflect
    cval=125,
    brightness_range=[0.7,1.0])



In [3]:
dir = 'srcImg/'

iterations = 5;

# read all classes
classes = os.listdir(dir)

In [4]:
def processImage(srcDir, cls, img, targetPath):
    iterations = 10
    i = 0
    try:
        path = srcDir+cls+'/'+img
        print(path)
        x = io.imread(path)
        x = x.reshape((1, ) + x.shape)
        for batch in datagen.flow(x, batch_size=16,
                          save_to_dir=targetPath,
                          save_prefix='aug-'+img,
                          save_format='png'):
            i+=1
            if i >= iterations: break;
    except BaseException as err:
        print(type(err))
        if(type(err) == KeyboardInterrupt):
            raise Exception("stopped process")
        print(f"Unexpected {err=}, {type(err)=}")

def processImageClass(cls, subThreads):
    print("starting with: "+cls)
    pool = mp.Pool(subThreads)
    targetPath = 'augmented/'+cls
    images = os.listdir(dir+cls)
    if not(os.path.exists(targetPath)): os.mkdir(targetPath)
    #for img in images:
    pool.starmap(processImage, zip(repeat(dir), repeat(cls), images, repeat(targetPath)))
    #pool.starmap(processImage, [dir, cls, (images), targetPath])
        #processImage(dir, cls, img, 'hello', targetPath)
    pool.close()
            
    print("done with: "+cls)

In [ ]:
#define pool for parallel processing
processors = mp.cpu_count()
augmentedPath = 'augmented'
if not(os.path.exists(augmentedPath)): os.mkdir(augmentedPath)
for cls in classes:
    processImageClass(cls, processors)
#pool.starmap(processImageClass, [(classes), subThreads])
#pool.starmap(processImageClass, zip(classes, repeat(subThreads)))

print('DONE!!!')


starting with: Amrit
srcImg/Amrit/ssssssss.png
srcImg/Amrit/WhatsApp Image 2021-11-11 at 18.03.10 (1).jpeg
srcImg/Amrit/WhatsApp Image 2021-11-11 at 18.03.12 (1).jpeg
srcImg/Amrit/WhatsApp Image 2021-11-11 at 18.03.16.jpeg
srcImg/Amrit/WhatsApp Image 2021-11-11 at 18.03.13 (1).jpeg
srcImg/Amrit/WhatsApp Image 2021-11-11 at 18.03.16 (2).jpeg
srcImg/Amrit/WhatsApp Image 2021-11-11 at 18.03.17 (2).jpeg
srcImg/Amrit/WhatsApp Image 2021-11-11 at 18.03.19.jpeg
srcImg/Amrit/WhatsApp Image 2021-11-11 at 18.03.17 (1).jpeg
srcImg/Amrit/WhatsApp Image 2021-11-11 at 18.03.14.jpeg
srcImg/Amrit/WhatsApp Image 2021-11-11 at 18.03.20.jpeg
srcImg/Amrit/WhatsApp Image 2021-11-11 at 18.03.13.jpeg
srcImg/Amrit/WhatsApp Image 2021-11-11 at 18.03.21 (1).jpeg
srcImg/Amrit/WhatsApp Image 2021-11-11 at 18.03.23 (1).jpeg
srcImg/Amrit/WhatsApp Image 2021-11-11 at 18.03.24 (1).jpeg
srcImg/Amrit/WhatsApp Image 2021-11-11 at 18.04.50 (2).jpeg
srcImg/Amrit/WhatsApp Image 2021-11-11 at 18.04.51.jpeg
srcImg/Amrit/What